In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from rl.agents import DQNAgent
import gym
from gym import spaces
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from keras.callbacks import TensorBoard
# from rl.callbacks import Callback
import os
import time

In [3]:
years_files = {
    # 2021: "../Data/merged_2021.xlsx",
    2022: "../Data/merged_2022.xlsx",
    2023: "../Data/merged_2023.xlsx"
}
dfs = []

for year, file_path in years_files.items():
    df = pd.read_excel(file_path, engine='openpyxl')
    df['Year'] = year
    dfs.append(df)
    
data = pd.concat(dfs, ignore_index=True)
pop_data = pd.read_excel('../Data/state_level_population_projection_wide_xgboost.xlsx')
pop_data_melted = pop_data.melt(
    id_vars=['Variant', 'Variant Description', 'Year', 'Age'],
    value_vars=['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
               'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
               'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
               'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'],
    var_name='state',
    value_name='population'
)
merged_data_population = pd.merge(
    data,
    pop_data_melted,
    left_on=['state', 'Year'],
    right_on=['state', 'Year'],
    how='left'
)

In [4]:
label_encoders = {}
categorical_cols = ['state', 'MDC', 'Age']
for col in categorical_cols:
    le = LabelEncoder()
    merged_data_population[col + '_encoded'] = le.fit_transform(merged_data_population[col].astype(str))
    label_encoders[col] = le  # Save encoders for later use
# Normalize numerical columns (e.g., 'total_patients')
scaler = StandardScaler()
numerical_cols = merged_data_population.columns.difference(['state', 'MDC','Variant','Variant Description','Fachabteilung', 'Sekundär','state_encoded','MDC_encoded','Land_x','Land_y'])
numerical_cols = [str(col) for col in numerical_cols]
merged_data_population[numerical_cols] = scaler.fit_transform(merged_data_population[numerical_cols])

C:\Users\Ali\AppData\Local\Temp\ipykernel_16552\2987491451.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data_population[col + '_encoded'] = le.fit_transform(merged_data_population[col].astype(str))
C:\Users\Ali\AppData\Local\Temp\ipykernel_16552\2987491451.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data_population[col + '_encoded'] = le.fit_transform(merged_data_population[col].astype(str))
C:\Users\Ali\AppData\Local\Temp\ipykernel_16552\2987491451.py:5: PerformanceWarning: DataFrame is highly fragm

In [5]:
merged_data_population[merged_data_population.select_dtypes(include='number').columns] = merged_data_population.select_dtypes(include='number').fillna(0)
merged_data_population[merged_data_population.select_dtypes(include='object').columns] = merged_data_population.select_dtypes(include='object').fillna("unknown")

In [6]:
state_columns = []
state_columns.extend(numerical_cols)
state_columns.extend(['state_encoded','total_patients'])
len(state_columns)

200

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

class iFCAAccessibility:
    """
    Integrated Floating Catchment Area (iFCA) implementation for hospital accessibility
    """
    
    def __init__(self, distance_threshold=50, distance_decay_function='gaussian', beta=1.0):
        """
        Initialize iFCA calculator
        
        Args:
            distance_threshold: Maximum distance to consider (km)
            distance_decay_function: 'linear', 'exponential', 'gaussian', or 'power'
            beta: Decay parameter for distance functions
        """
        self.distance_threshold = distance_threshold
        self.distance_decay_function = distance_decay_function
        self.beta = beta
        
    def _calculate_distance_decay(self, distances):
        """Calculate distance decay weights"""
        # Mask distances beyond threshold
        distances = np.where(distances <= self.distance_threshold, distances, np.inf)
        
        if self.distance_decay_function == 'linear':
            weights = np.where(distances != np.inf, 
                             1 - (distances / self.distance_threshold), 0)
        elif self.distance_decay_function == 'exponential':
            weights = np.where(distances != np.inf, 
                             np.exp(-self.beta * distances), 0)
        elif self.distance_decay_function == 'gaussian':
            weights = np.where(distances != np.inf, 
                             np.exp(-0.5 * (distances / self.beta) ** 2), 0)
        elif self.distance_decay_function == 'power':
            weights = np.where(distances != np.inf, 
                             distances ** (-self.beta), 0)
        else:
            # Default to inverse distance
            weights = np.where(distances != np.inf, 1 / (distances + 1), 0)
            
        return weights
    
    def compute_ifca_accessibility(self, hospitals_df, population_df, 
                                  hospital_coords_cols=['lat', 'lon'],
                                  population_coords_cols=['lat', 'lon'],
                                  hospital_capacity_col='total_beds',
                                  population_demand_col='population',
                                  service_type_col=None):
        """
        Compute iFCA accessibility scores
        
        Args:
            hospitals_df: DataFrame with hospital data
            population_df: DataFrame with population/demand data
            hospital_coords_cols: Column names for hospital coordinates
            population_coords_cols: Column names for population coordinates
            hospital_capacity_col: Column name for hospital capacity (beds)
            population_demand_col: Column name for population/demand
            service_type_col: Column name for service type matching (optional)
        
        Returns:
            accessibility_scores: Array of accessibility scores for each population location
        """
        hospital_coords = hospitals_df[hospital_coords_cols].values
        population_coords = population_df[population_coords_cols].values

        distance_matrix = cdist(population_coords, hospital_coords, metric='euclidean')
        
        distance_matrix = distance_matrix * 111  
        hospital_capacity = hospitals_df[hospital_capacity_col].values
        population_demand = population_df[population_demand_col].values
        
        hospital_ratios = np.zeros(len(hospitals_df))
        
        for j, hospital_cap in enumerate(hospital_capacity):
            distances_to_hospital = distance_matrix[:, j]
            weights = self._calculate_distance_decay(distances_to_hospital)
        
            weighted_demand = np.sum(population_demand * weights)

            if weighted_demand > 0:
                hospital_ratios[j] = hospital_cap / weighted_demand
            else:
                hospital_ratios[j] = 0

        accessibility_scores = np.zeros(len(population_df))
        
        for i in range(len(population_df)):
            distances_to_hospitals = distance_matrix[i, :]
            weights = self._calculate_distance_decay(distances_to_hospitals)
            accessibility_scores[i] = np.sum(hospital_ratios * weights)
        
        return accessibility_scores

def compute_iFCA_accessibility(hospitals_data, population_data=None, 
                              mdc_type=None, current_patient_state=None,
                              mdc_to_fachabteilung=None):

    ifca_calc = iFCAAccessibility(distance_threshold=100, 
                                 distance_decay_function='gaussian', 
                                 beta=20)

    hospitals_df = hospitals_data.copy()
    
    dept_columns = [col for col in hospitals_df.columns if col.startswith(('0', '1', '2', '3', '8', '9'))]
    
    # Calculate service-specific beds if MDC is provided
    if mdc_type and mdc_to_fachabteilung and mdc_type in mdc_to_fachabteilung:
        # Get relevant departments for this MDC
        mapping = mdc_to_fachabteilung[mdc_type]
        primary_depts = mapping.get("Fachabteilung", [])
        secondary_depts = mapping.get("Sekundär", [])
        
        # Find available departments in the data
        relevant_depts = []
        for dept in primary_depts + secondary_depts:
            # Check if department exists in data (with or without leading zeros)
            dept_variations = [dept, dept.lstrip('0'), f"{int(dept):04d}"]
            for variation in dept_variations:
                if variation in dept_columns:
                    relevant_depts.append(variation)
                    break
        
        if relevant_depts:
            # Calculate beds for relevant departments only
            hospitals_df['service_beds'] = hospitals_df[relevant_depts].sum(axis=1, skipna=True)
            capacity_col = 'service_beds'
        else:
            # Fallback to total beds if no specific departments found
            hospitals_df['total_beds'] = hospitals_df[dept_columns].sum(axis=1, skipna=True)
            capacity_col = 'total_beds'
    else:
        # Calculate total available beds across all departments
        hospitals_df['total_beds'] = hospitals_df[dept_columns].sum(axis=1, skipna=True)
        capacity_col = 'total_beds'
    
    # Add dummy coordinates if not present (replace with actual coordinates)
    if 'lat' not in hospitals_df.columns:
        np.random.seed(42)  # For reproducibility
        hospitals_df['lat'] = np.random.uniform(47, 55, len(hospitals_df))  # Germany latitude range
        hospitals_df['lon'] = np.random.uniform(6, 15, len(hospitals_df))   # Germany longitude range
    
    # Prepare population data
    if population_data is None:
        # Use the same data as population centers
        population_df = hospitals_data.copy()
        population_df['population'] = population_df.get('population', 1000)  # Default population
    else:
        population_df = population_data.copy()
    
    # Add dummy coordinates for population if not present
    if 'lat' not in population_df.columns:
        np.random.seed(123)
        population_df['lat'] = np.random.uniform(47, 55, len(population_df))
        population_df['lon'] = np.random.uniform(6, 15, len(population_df))
    
    # Filter hospitals with capacity > 0
    capacity_mask = hospitals_df[capacity_col] > 0
    hospitals_df = hospitals_df[capacity_mask]
    
    # Ensure we have valid data
    if len(hospitals_df) == 0 or hospitals_df[capacity_col].sum() == 0:
        return np.array([0.0] * len(population_df))
    
    # Compute accessibility
    try:
        accessibility_scores = ifca_calc.compute_ifca_accessibility(
            hospitals_df=hospitals_df,
            population_df=population_df,
            hospital_capacity_col=capacity_col,
            population_demand_col='population'
        )
        
        return accessibility_scores
        
    except Exception as e:
        print(f"Error computing iFCA accessibility: {e}")
        return np.array([0.0] * len(population_df))




In [25]:
# Modified HospitalEnv class methods to integrate iFCA
class HospitalEnvWithiFCA:
    """
    Modified HospitalEnv class with proper iFCA integration
    """
    
    def __init__(self, data, mdc_to_fachabteilung):
        # ... (keep your existing initialization code)
        self.data = data
        self.num_beds = 6
        self.state_size = 200
        self.mdc_to_fachabteilung = mdc_to_fachabteilung
        
        # Get all department columns (updated to include all department codes)
        self.dept_columns = [col for col in data.columns if col.startswith(('0', '1', '2', '3', '8', '9'))]
        
        # Initialize iFCA calculator
        self.ifca_calculator = iFCAAccessibility(
            distance_threshold=100,
            distance_decay_function='gaussian',
            beta=20
        )
        
        # Calculate initial accessibility baseline
        self.previous_accessibility = self._compute_current_accessibility()
    
    def _compute_current_accessibility(self, mdc_type=None):
        """Compute accessibility for current hospital configuration"""
        try:
            # Use the updated compute_iFCA_accessibility function
            accessibility_scores = compute_iFCA_accessibility(
                hospitals_data=self.data,
                mdc_type=mdc_type,
                mdc_to_fachabteilung=self.mdc_to_fachabteilung
            )
            
            return accessibility_scores
            
        except Exception as e:
            print(f"Error computing accessibility: {e}")
            return np.array([0.0])
    
    def _reallocate_beds_with_mapping(self, mdc_index, beds):
        """
        Reallocate beds using the MDC to department mapping
        """
        # Get MDC from index
        mdc_list = list(self.mdc_to_fachabteilung.keys())
        if mdc_index >= len(mdc_list):
            return
        
        mdc_label = mdc_list[mdc_index]
        
        # Get department mapping
        mapping = self.mdc_to_fachabteilung.get(mdc_label, {})
        primary_depts = mapping.get("Fachabteilung", [])
        secondary_depts = mapping.get("Sekundär", [])
        
        # Find available departments in current data
        available_depts = []
        for dept in primary_depts + secondary_depts:
            # Check different variations of department codes
            dept_variations = [dept, dept.lstrip('0'), f"{int(dept):04d}"]
            for variation in dept_variations:
                if variation in self.dept_columns:
                    available_depts.append(variation)
                    break
        
        if len(available_depts) < 2 or beds == 0:
            return
        
        # Get current utilization for available departments
        current_utilization = {}
        for dept in available_depts:
            current_utilization[dept] = self.current_patient[dept].iloc[0] if dept in self.current_patient.columns else 0
        
        # Find source (highest utilization) and target (lowest utilization) departments
        if current_utilization:
            source_dept = max(current_utilization, key=current_utilization.get)
            target_dept = min(current_utilization, key=current_utilization.get)
            
            # Reallocate beds
            if source_dept in self.current_patient.columns and target_dept in self.current_patient.columns:
                current_source = self.current_patient[source_dept].iloc[0]
                self.current_patient.loc[self.current_patient.index[0], source_dept] = max(0, current_source - beds)
                self.current_patient.loc[self.current_patient.index[0], target_dept] += beds
    
    
    def _get_state(self):
        # Extract ONLY the two features we need
        # print(self.current_patient[numerical_cols])
        out = self.current_patient[state_columns].values.flatten()
        
        return out
    
    def reset(self):
        # Randomly select a patient case
        self.current_patient = self.data.sample(1)
        return self._get_state()
    
    def _calculate_reward(self, new_accessibility=None):
        """Calculate reward based on accessibility improvement"""
        if new_accessibility is None:
            new_accessibility = self._compute_current_accessibility()
        
        # Calculate improvement in mean accessibility
        current_mean = np.mean(new_accessibility)
        previous_mean = np.mean(self.previous_accessibility)
        
        reward = current_mean - previous_mean
        
        # Update previous accessibility
        self.previous_accessibility = new_accessibility.copy()
        
        return reward
    
    def decode_action(self, action):
        mdc = action // self.num_beds
        beds = action % self.num_beds
        return mdc, beds
    
    def step(self, action):
        """Modified step function with iFCA integration"""
        # Decode and apply action
        mdc_index, beds = self.decode_action(action)
        
        # Use the improved bed reallocation with mapping
        self._reallocate_beds_with_mapping(mdc_index, beds)
        
        # Get the MDC type for service-specific accessibility calculation
        mdc_list = list(self.mdc_to_fachabteilung.keys())
        current_mdc = mdc_list[mdc_index] if mdc_index < len(mdc_list) else None
        
        # Compute new accessibility (service-specific)
        new_accessibility = self._compute_current_accessibility(mdc_type=current_mdc)
        
        # Calculate reward
        reward = self._calculate_reward(new_accessibility)
        
        # Get next state
        next_state = self._get_state()
        
        # Check if episode is done
        done = False  # Define your termination condition
        
        info = {
            "accessibility": new_accessibility,
            "mean_accessibility": np.mean(new_accessibility),
            "accessibility_std": np.std(new_accessibility),
            "mdc_type": current_mdc,
            "beds_reallocated": beds
        }
        
        return next_state, reward, done, info


In [ ]:
from mdc_mapping import mdc_to_fachabteilung
class ReshapeProcessor(Processor):
    def process_state_batch(self, state_batch):
        # Remove the extra dimension: (batch, 1, 2) → (batch, 2)
        return np.squeeze(state_batch, axis=1)

def Main(merged_data_population, mdc_to_fachabteilung):
    
    env = HospitalEnvWithiFCA(merged_data_population, mdc_to_fachabteilung)
    mdc_list = list(mdc_to_fachabteilung.keys())
    n_actions = len(mdc_list) * 6  # 6 beds per action
    
    model = Sequential([
        Dense(128, activation='relu', input_shape=(200,)),
        Dense(128, activation='relu'),
        Dense(n_actions, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # Create agent with updated action space
    policy = EpsGreedyQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    
    agent = DQNAgent(
        model=model,
        policy=policy,
        memory=memory,
        nb_steps_warmup=100,
        nb_actions=n_actions,
        batch_size=32,
        enable_double_dqn=True,
        processor=ReshapeProcessor()
    )
    
    agent.compile(optimizer='adam')
    
    return env, agent

env,agent = Main(merged_data_population,mdc_to_fachabteilung)

Testing iFCA with MDC Mapping:
Overall Accessibility - Mean: 0.0073
01 - Nervensystem - Mean: 0.0018
05 – Kreislaufsystem - Mean: 0.0016
08 – Muskel-Skelett-System & Bindegewebe - Mean: 0.0039


In [27]:
from rl.callbacks import Callback
import tensorflow as tf
class CustomTensorBoardCallback(Callback):
    def __init__(self, log_dir="./logs/"):
        self.writer = tf.summary.create_file_writer(log_dir)
        self.step = 0

    def on_step_end(self, step, logs={}):
        self.step += 1
        with self.writer.as_default():
            if 'reward' in logs:
                tf.summary.scalar('reward', logs['reward'], step=self.step)
            if 'loss' in logs:
                tf.summary.scalar('loss', logs['loss'], step=self.step)
            self.writer.flush()


In [ ]:
import warnings
mdc_list = list(mdc_to_fachabteilung.keys())
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    agent.fit(env, nb_steps=10000, visualize=False, callbacks=[CustomTensorBoardCallback()])

In [ ]:
env = HospitalEnv(data)
state = env.reset()
print("State shape:", state.shape)  # Should be (2,)
print("State values:", state)

In [ ]:
scores = agent.test(env, nb_episodes=100, visualize=False)
print("Average reward:", np.mean(scores.history['episode_reward']))

In [46]:
test_obs = env.reset()
action = agent.forward(test_obs)
print(f"Recommended Department: {label_encoders['MDC'].inverse_transform([action])}")

Recommended Department: ['-1 – Fehler-DRGs und sonstige DRGs']


In [22]:
print(action)

0


In [44]:
!tensorboard --logdir=./logs

^C
